### 구글 코랩 T5 라지모델

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 기본 작업 경로 설정
import os
notebook_path = os.path.abspath("drive/MyDrive/kdt_project_3/readme.md")
notebook_dir = os.path.dirname(notebook_path)
os.chdir(notebook_dir)

# 현재 작업 디렉토리 출력
print("Current working directory: ", os.getcwd())

Current working directory:  /content/drive/MyDrive/kdt_project_3


In [3]:
import json
import random

# JSON 파일에서 딕셔너리 읽기
with open('data/text_data/output_text.json', 'r') as file:
    data_loaded = json.load(file)

# 데이터 섞기
items = list(data_loaded.items())
random.shuffle(items)

# 데이터를 섞은 후 딕셔너리로 변환
data_loaded_shuffled = dict(items)


['공포, 헤드폰', '아찔한 분위기네요! 🎧 어떤 음악 듣고 계신가요? 궁금해요! 😊']

In [4]:
# 학습 및 테스트 데이터셋 나누기
train_x = [data_loaded_shuffled[i][0] for i in data_loaded_shuffled][:5000]
train_y = [data_loaded_shuffled[i][1] for i in data_loaded_shuffled][:5000]
test_x = [data_loaded_shuffled[i][0] for i in data_loaded_shuffled][5000:]
test_y = [data_loaded_shuffled[i][1] for i in data_loaded_shuffled][5000:]

In [6]:
!pip install transformers --upgrade
!pip install datasets

In [7]:
from transformers import T5Config ,T5TokenizerFast, T5ForConditionalGeneration, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import Dataset
from tqdm import tqdm
import subprocess



name_folder = 'transfer_1'
server_port = '4561'
batch_size = 16
train_epochs = 60



# TensorBoard 서버 실행 (백그라운드)
subprocess.Popen(['tensorboard', f'--logdir=t5/{name_folder}', f'--port={server_port}'])

<Popen: returncode: None args: ['tensorboard', '--logdir=t5/transfer_1', '--...>

### 코랩 에서는 large 모델 시도 인자 조정
- 드롭 아웃 비율 조정 기본값 : 0.1 -> 0.2
- 학습 률 조정
- 웜 업 스텝 설정
- 배치 사이즈 : 코랩에서 돌릴때 시도
- l2 정규화 (weight decay)
- gredient clipping 그라디언트 조정으로 학습 안정화 시키기
- 레이블 스무딩

In [8]:
# 모델과 토크나이저 로드
tokenizer = T5TokenizerFast.from_pretrained('paust/pko-t5-large')
config = T5Config.from_pretrained('paust/pko-t5-large')
model = T5ForConditionalGeneration.from_pretrained('paust/pko-t5-large')

config.dropout_rate = 0.2  # 드롭아웃 비율을 20%로 설정 (기본값은 0.1)


# 데이터 전처리 함수 정의
def preprocess_function(examples):
    model_inputs = tokenizer(examples['input_text'], max_length=128, truncation=True, padding='max_length')
    labels = tokenizer(examples['target_text'], max_length=128, truncation=True, padding='max_length')
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# 데이터셋 생성 및 토큰화
dataset_train = Dataset.from_dict({'input_text': train_x,'target_text': train_y})
dataset_test = Dataset.from_dict({'input_text': test_x,'target_text': test_y})

tokenized_train_datasets = dataset_train.map(preprocess_function, batched=True)
tokenized_test_datasets = dataset_test.map(preprocess_function, batched=True)

# 학습인자
training_args = TrainingArguments(
    per_device_train_batch_size=batch_size, # 학습 배치 사이즈
    per_device_eval_batch_size=batch_size,  # 평가 배치 사이즈
    output_dir=f't5/{name_folder}',         # 모델 및 체크포인트 저장 디렉토리
    num_train_epochs=train_epochs,          # 학습 에폭 수
    logging_dir=f't5/{name_folder}/logs',   # TensorBoard 로그가 저장될 디렉토리
    logging_steps=100,                      # TensorBoard 로그를 기록할 간격
    report_to='tensorboard',                # TensorBoard로 로깅
    load_best_model_at_end = True,
    eval_strategy='epoch',
    save_strategy='epoch',                  # 에포크 마다 모델 저장
    # resume_from_checkpoint=True           # 이어 학습

    # 추가 인자
    learning_rate=5e-5,                     # 기본값에서 시작
    lr_scheduler_type="linear",             # 스케줄러
    warmup_steps=500,                       # 500 스텝 동안 학습률을 점진적으로 증가
    weight_decay=0.01,                      # l2 정규화 기법 중 하나
    max_grad_norm=1.0,                      # 그라디언트 클리핑
)

# Trainer 객체 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_test_datasets,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=10)]
)

# 모델 학습
trainer.train() # 이어 학습시 (resume_from_checkpoint=checkpoint_dir)

# 모델과 토크나이저 저장
model_save_path = f't5/{name_folder}/model'
tokenizer_save_path = f't5/{name_folder}/model/tokenizer'

model.save_pretrained(model_save_path)
tokenizer.save_pretrained(tokenizer_save_path)

print(f"모델이 '{model_save_path}'에 저장되었습니다.")
print(f"토크나이저가 '{tokenizer_save_path}'에 저장되었습니다.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.94k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/994 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,0.481000,0.253806
2,0.220000,0.193124
3,0.194600,0.181576
4,0.176800,0.178080
5,0.168300,0.175399
6,0.158400,0.174484
7,0.150800,0.175241
8,0.144800,0.175933
9,0.136900,0.179480
10,0.132400,0.182000


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


모델이 't5/transfer_1/model'에 저장되었습니다.
토크나이저가 't5/transfer_1/model/tokenizer'에 저장되었습니다.


### 결과 확인

In [2]:
# 기본 작업 경로 설정
import os
notebook_path = os.path.abspath("../readme.md")
notebook_dir = os.path.dirname(notebook_path)
os.chdir(notebook_dir)

# 현재 작업 디렉토리 출력
print("Current working directory: ", os.getcwd())

Current working directory:  /mnt/e/py_data/project_3_git


In [4]:
model = T5ForConditionalGeneration.from_pretrained('models/t5/transfer_1_large/model/')
tokenizer = T5TokenizerFast.from_pretrained('models/t5/transfer_1_large/model/tokenizer/')

/home/tommy/miniconda3/envs/p3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-09-12 09:48:14.122834: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-12 09:48:14.257843: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-12 09:48:14.295455: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-12 09:48:14.614233: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow bin

In [18]:
import torch

# 테스트 입력
test_input = "행복, 강아지"

# 입력 토큰화
input_ids = tokenizer.encode(test_input, return_tensors='pt')

# GPU가 사용 가능한지 확인하고, 사용 가능한 경우 GPU로 이동
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델과 토크나이저를 GPU로 이동
model.to(device)

# 입력 데이터도 GPU로 이동
input_ids = input_ids.to(device)

# 예측을 GPU에서 수행
with torch.no_grad():
    outputs = model.generate(
    input_ids, 
    max_length=50, 
    num_return_sequences=1,  # 여러 샘플을 원할 경우 1보다 크게 설정
    do_sample=True,          # 샘플링 모드로 전환
    top_k=50,                # 상위 k개 샘플 중에서 선택
    top_p=0.95,              # 누적 확률이 0.95에 도달할 때까지 샘플 선택
    temperature=1.0          # 낮은 온도는 더 결정적, 높은 온도는 더 랜덤한 출력
)

# 예측 결과 디코딩
predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"테스트 입력: {test_input}")
print(f"모델의 예측: {predicted_text}")

테스트 입력: 행복, 강아지
모델의 예측: 행복한 표정이네요! 😄 강아지도 정말 귀여워요!
